# Проект 10 создание модели для прогнозирования страховых случаев

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score

data = pd.read_csv('/datasets/insurance.csv')

data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [2]:
data['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

## 2. Умножение матриц

In [3]:
features = data.drop(['Страховые выплаты'] , axis=1)
target = data['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

def create_invertible_matrix(size):
    n = 0
    while n <= 2:
        try: 
            n += 1
            random_matrix = np.random.normal(size=(size,size))
            invertible_matrix = np.linalg.inv(random_matrix) 
        except: #любая матрица, которая является обратной для какой-то матрицы, является обратимой
            n = 0
    return invertible_matrix
#
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [4]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w =  np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        self.full_w = w
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
    def return_w(self):
        return self.w
    
    def return_w0(self):
        return self.w0
    
    def return_full_w(self):
        return self.full_w    
#
model = LinearRegression()
model.fit(features_train, target_train)

predictions = model.predict(features_valid)
r2_X = r2_score(target_valid, predictions)
print(r2_X)
w_model_1 = model.return_full_w()
print('w =', w_model_1)

0.43522757127026657
w = [-9.32136675e-01  1.79258369e-02  3.57228278e-02 -5.46000708e-07
 -1.26186590e-02]


In [5]:
random_invertible_matrix = create_invertible_matrix(features_train.shape[1])
XP = np.dot(features_train, random_invertible_matrix)
model.fit(features_train, target_train)

predictions = model.predict(features_valid)
r2_X_dot_P = r2_score(target_valid, predictions)
print(r2_X_dot_P)

w_model_2 = model.return_full_w()
print('w =', w_model_2)

0.43522757127026657
w = [-9.32136675e-01  1.79258369e-02  3.57228278e-02 -5.46000708e-07
 -1.26186590e-02]


In [18]:
features_valid

,Пол,Возраст,Зарплата,Члены семьи
3183,0,33.0,39000.0,4
1071,0,50.0,43100.0,2
2640,1,39.0,42100.0,0
2282,0,20.0,34800.0,0
1595,0,41.0,40000.0,4
...,...,...,...,...
982,1,51.0,29000.0,2
3820,1,33.0,57900.0,3
3595,1,35.0,42300.0,0
3513,1,36.0,41300.0,0


In [42]:
features_vector_id = features_valid.loc[features_valid.index == 982].values
w = w_model_1[1:]
w0 = w_model_1[0]
otvet = float(features_vector_id.dot(w) + w0) #round
print(otvet)
#print(features_vector_id)

0.8665820388884379


# Преобразование данных:

Дано задание:

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [9]:
encoding_vector = []
for column in features.columns:
    max_column = features[column].max()/1000
    encoding_vector.append(max_column)
#

encoding_features = (features/encoding_vector)+0.01
encoding_features_train, encoding_features_valid, target_train, target_valid = train_test_split(encoding_features, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(encoding_features_train, target_train)

predictions = model.predict(encoding_features_valid)
r2_encoding = r2_score(target_valid, predictions)
print('r2 для закодированного дата сета =',r2_encoding)

r2 для закодированного дата сета = 0.43522757127026734


Значение r2 почти не поменялось, хотя персональные данные теперь выглядят так:

In [10]:
encoding_features

,Пол,Возраст,Зарплата,Члены семьи
0,1000.01,630.779231,627.858101,166.676667
1,0.01,707.702308,481.022658,166.676667
2,0.01,446.163846,265.832785,0.010000
3,0.01,323.086923,527.858101,333.343333
4,1000.01,430.779231,330.389747,0.010000
...,...,...,...,...
4995,0.01,430.779231,451.908734,333.343333
4996,0.01,523.086923,663.301139,166.676667
4997,0.01,307.702308,429.123924,333.343333
4998,1000.01,338.471538,413.934051,500.010000


Такой метод преобразования выбран, чтобы отмаштабировать все данные в диапазон от 0 до 1, проще всего это сделать путём деления на максимальное значение в данном столбце, при этом не уменьшиться качество обучени и предсказания машинного обучения, так как все данные относительно друг друга сохраняют единое отностильное расстояние.

Комментарий к преобразованию данных для защиты персональной информации: данные закодированы, так, что сложно их расшифровать, чтобы узнать реальные доходы, возраст или количество членов семьи у клиента.

При желании декодировать и получить персональную информацию - можно, но даже Энигму удалось дешифровать в польском Бюро шифров в декабре 1932 года. ( Энигма - это переносная шифровальная машина, использовавшаяся для шифрования и дешифрования секретных сообщений, применявшаяся с 20-х годов XX века, наибольшее распространение получила в нацистской Германии во время Второй мировой войны)

Но учитывая, что r2 не меняется, если умножить признаки, на любую обратимую матрицу, лучше закодировать данные как X (матрица признаков, умноженная на случайную обратимую матрицу

Получаются данные, которые очень сложно расшифровать, и которые скрывают персональные данные:

In [11]:
XP_data = pd.DataFrame(data = XP, columns = ['Пол','Возраст','Зарплата','Члены семьи'])
XP_data

,Пол,Возраст,Зарплата,Члены семьи
0,-27120.584597,-12946.317435,-55387.439315,-2193.949385
1,-43148.254598,-20584.089818,-88082.047492,-3468.072874
2,-30789.825277,-14690.875895,-62859.251071,-2479.313717
3,-33786.498471,-16120.987871,-68978.186429,-2721.199169
4,-37906.016221,-18084.123156,-77379.740745,-3048.127642
...,...,...,...,...
3745,-24049.686722,-11480.872100,-49118.117182,-1947.652978
3746,-17007.984220,-8119.576749,-34731.260699,-1375.935076
3747,-33487.046894,-15980.825913,-68375.688087,-2701.038467
3748,-37530.525199,-17901.594803,-76599.879516,-3010.846174


# Сохранённые комментарии ревьюера:

Стас, добрый день! Это Сослан, я уже делал ревью двух Ваших работ. 

Этот проект немного отличается от всех остальных в курсе, поскольку содержит большую теоретическую (математическую) часть. Поэтому когда у Вас просят во втором пункте задания "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?" речь идет именно об математическом обосновании. 

Вы можете взять произвольную обратимую матрицу (сгенерировать ее, например, случайно) и умножить признаки. После этого Вам нужно обучить две модели с признаками без умножения на матрицу и с умножением на матрицу и сравнить результаты регрессии.

Но то же самое можно установить строго математически если поиграться с формулой регрессии:
$$
w = (X^T X)^{-1} X^T y
$$

здесь у нас X это признаки, а $Xw$ это прогнозы. Можно вместо $X$ подставить в формулу $XP$ - ($P$ это у нас обратимая матрица) и посмотреть что случится с новыми коэффициентами $w$ и новым прогнозом.

В принципе, достаточно проверить на одном частном случае программно, но если у Вас получится это сделать в общем будет совсем круто.

---

По третьему пункту, хоть масштабирование признаков действительно никак не повлияет на результат регрессии, но данные воссстанавливаются довольно легко. Пол не поменялся совсем, а количество членов семьи можно узнать на глазок просто умножив значения на 1, 2, 3 ... пока все значения не станут целыми. Про алгоритм у Вас есть подсказка во втором пункте, нужно ей воспользоваться :).

---

Попробуйте проработать проект в этом ключе, если у Вас возникнут вопросы, можете задавать их мне со следующей итерацией. Мне кажется, будет полезным еще раз просмотреть разделы спринта посвященные матрицам и линейной регрессии:

https://praktikum.yandex.ru/learn/data-scientist/courses/16e4a25a-c348-4de0-9c66-9a7ae91fdb8a/sprints/566/topics/99e9532a-ff23-40ed-9d87-ef2a892f518c/lessons/9b8918c1-0ae2-4995-b072-f0236a24aa3e/

https://praktikum.yandex.ru/learn/data-scientist/courses/16e4a25a-c348-4de0-9c66-9a7ae91fdb8a/sprints/566/topics/221dc161-ee50-4915-8371-e6f3434c874b/lessons/f439ad72-d7c0-4fcb-838b-b2c16c684a75/м

---
Комментарий от ученика:

Да, я дебил, я перепутал термины обратная и обратимая матрица... Из за этого не правильно понял задание, ну и соответственно не правильно всё сделал. Одногруппники из кагорты в слаке объяснили мою ошибку. Коллективный разум сила!

Касательно формулы - я не представляю как это сделать, тем более учитывая что в переменных матрица, которая ещё и транспонированная, я плохо знаю высшую математику.

Поэтому выбрал вариант умножить на случайную обратимую матрицу.

Вариант с зашифровкой данных в датасете - переделал.

---

Все нормально, кажется нам осталась одна итерация. Не буду Вас мучать математикой, тем более что Вы почти выписали ее в коде :). Посмотрите пожалуйста то, что я добавил в Ваш проект и попробуйте все же дожать алгоритм.

---

UPD Комментарий от ученика:

Переделал код с учётом комментариев. Понимаю, что можно было не писать второй класс модели в таком случае, а лишь на вход в стандартную модель обучение подать матрицу признаков (train_features) умноженную на P (случайную матрицу), но так как я уже делал специальный класс, я решил доделать начатое, для большей автоматизации. Тем более мне было интересно писать данный код: особенно конструкцию с while и try/except

---

У меня поломался код в тренажере. Обратите внимание, что при правильной реализации умножение на обратимую матрицу не должно изменить результата регрессии (это и есть фишка данного задания :)

---

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

---

UPD комментирий от ученика:

Пытался понять в чём ошибка переделывая свой класс модели 100500 раз, в итоге получался отрицательный r2, не понял в чём ошибка, в итоге решил пойти по простому рекомендованному методу, переобучил модель ещё раз, вместо X обучил модель на XP. Так же изменил способ кодирования данных.

Спасибо за терпение в проверках моих проектов, Сослан, ваши комментарии позволяют усваивать материал намного лучше, я вам очень благодарен. По факту это не большой проект, и весь проект можно было сделать за 2 часа, я сперва не сразу хорошо усвоил материал, но переделывая проект - усвоил его намного лучше. 

---

Спасибо за добрые слова :). Мне бывает очень приятно, когда на мои замечания реагируют таким образом. Своими комментариями я пытаюсь помочь обучающемуся разобраться лучше в нюансах. Мне кажется это важно, а принять проект - проще всего. Что касается конкретно данного задания его целью является закрепление материалов связанных с линейными свойствами матриц. Я могу зачесть Вам проект в таком виде, единственное добавлю, что в качестве алгоритма можно просто рассмотреть умножение данных на произвольную обратимую матрицу. Потому что ранее мы установили, что вследствие такого умножения результат линейной регрессии не меняется. 

Успехов в дальнейшей учебе.

---

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** ...

**Обоснование:** ...

<div class="alert alert-danger" role="alert">
<s>Эта штука не совсем корректно работает, потому что умножать на обратимую матрицу нужно только признаки, а Вы добавили еще единичками свободный член и умножаете после этого. На самом деле не нужно писать две линейные регрессии из-за этого. Можно просто на вход подать другую матрицу уже умноженную на обратимую. Я там добавил ниже код использующий Вашу линейную регрессию и функцию получения обратимой матрицы. И если сравнить результат, он не изменится с точностью до погрешности. Что наводит нас на соответствующий алгоритм.</s></div>

<div class="alert alert-success" role="alert">
Исправлено</div>
---

Еще я обратил внимание, что когда Вы писали две разные регрессии, Вы почти выписали математическое доказательство, только в коде. Через $w_1$ я обозначил новые веса которые получатся при умножении наших признаков на обратимую матрицу $P$

Вот что у Вас получилось:

$$
w_1 = ((XP)^TXP)^{-1}(XP)^Ty
$$

если здесь аккуратно раскрыть скобки 

а мы помним из курса, что $(XP)^T = P^T X^T$, а для квадратных обратимых матриц $(AB)^{-1} = B^{-1}A^{-1}$ 

у нас получится

$$
w_1 = (P^TX^TXP)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty
$$

Здесь мы можем заменить $(X^TX)^{-1}X^Ty$ на $w$ по указанной выше формуле и как раз получим требуемое:

$$
w_1=P^{-1}w
$$

---